# TODO
- vP is not allow to be 0
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime
- t must not be prime --> optimise
- when there is no R large/ small en
- when delta of bounds is small then the possibility of a new generation of p is high 
- when p is smaller than there is is higher chance of success



In [499]:
MIN_PRIME = 0
MAX_PRIME = 10
OPT_RANGE_REDUCTION = 0

MAX_ITERS = 10



msg = 12345678900987654321

In [500]:
import sympy
import random

def genPrime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def genSlightlySmallerPrime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def genPrimeWithLimit(lb, ub):
    return sympy.randprime(lb, ub)

def genSmallPrime():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def fermatPrime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def mmi(E, phiN):
    return pow(E, -1, phiN)

def calcN(P, Q):
    return P*Q

def calcPhiN(P, Q):
    return (P-1) * (Q-1)

def toBinary(dec):
    return "{0:b}".format(dec)

def toDecimal(bin):
    return int(bin, 2)

def concatenateInBinary(prev, tail):
    prev = toBinary(prev)
    tail = toBinary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return toDecimal(prev + tail)

def splitInBinary(bin):
    bin = toBinary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(toDecimal(prev), toDecimal(tail))



# Voraussetzungen

In [501]:
def createAttackerKey():
    a_P = genSmallPrime()
    a_Q= genSmallPrime()


    a_PhiN = calcPhiN(a_P, a_Q)
    a_N = calcN(a_P, a_Q)

    a_E = fermatPrime()
    a_D = mmi(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

Attacker = {}

created = False
while not created:
    try:
        Attacker = createAttackerKey()
        created = True
    except ValueError:
            print("Trying to generate Key failed. Trying again ...")

print(Attacker)

{'P': 23, 'Q': 29, 'PhiN': 616, 'N': 667, 'E': 5, 'D': 493}


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [502]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
User = {}

valid = False
while not valid:
    # Schritt 1
    P = genSlightlySmallerPrime()
    vP = pow(P, Attacker["E"] ,Attacker["N"])

    # Schritt 2
    t = genPrime() 
    tempN = concatenateInBinary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    print("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(genPrimeWithLimit(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    User["P"] = P
    User["Q"] = Q


print("Amount of failed attempts",r_too_large)

Attacker["vP"] = vP
Attacker["R"] = R

print(R, Q)

print(vP, t)

print(tempN)

Attacker["vP"] = vP
Attacker["R"] = R

print("tempN = P * Q + R")
print(tempN, "=", P, "*", Q, "+", R)

Bounds: [5093, 5108] 	 P: 71 	 deltaB: 15
Amount of failed attempts 0
90 5107
354 191
362687
tempN = P * Q + R
362687 = 71 * 5107 + 90


## Schritt 4

In [503]:
N = calcN(User["P"], User["Q"])
User["N"] = N

print("vP||t:", toBinary(vP), toBinary(t))
print("N':   ",toBinary(tempN))

print("N :",toBinary(N))
print("N':",toBinary(tempN))

print(N)

vP||t: 101100010 10111111
N':    1011000100010111111
N : 1011000100001100101
N': 1011000100010111111
362597


## Schritt 5

In [504]:
def createUserKey():
    PhiN = calcPhiN(P, Q)
    E = fermatPrime()
    D = mmi(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        User.update(createUserKey())
        created = True
    except ValueError:
            print("Trying to generate Key failed. Trying again ...")
print("User Key successfully generated!")
print(User)

Trying to generate Key failed. Trying again ...
User Key successfully generated!
{'P': 71, 'Q': 5107, 'N': 362597, 'PhiN': 357420, 'E': 17, 'D': 147173}


## Schritt 6

In [505]:
cipher = pow(msg, User["E"], User["N"])
signature = pow(msg, User["D"], User["N"])

# Angriff

## Schritt 1

In [506]:
pub_User = {
    "N" : User["N"],
    "E" : User["E"]
}

print(pub_User)

collection_User = {
    "N" : pub_User["N"],
    "E" : pub_User["E"]
}

{'N': 362597, 'E': 17}


## Schritt 2

In [507]:
exp_vP = splitInBinary(User["N"])[0]

print("Known vP:",toBinary(Attacker["vP"]))
print("Calc vP: ",toBinary(exp_vP))
print(toBinary(User["N"]))


print(toBinary(Attacker["R"]))
print(Attacker["R"])




Known vP: 101100010
Calc vP:  101100010
1011000100001100101
1011010
90


## Schritt 3

In [508]:
exp_P = pow(exp_vP, Attacker["D"], Attacker["N"])

exp_alt_P = pow(exp_vP+1, Attacker["D"], Attacker["N"])

print(exp_P, exp_alt_P)
print("Actual value of P:", User["P"])

71 227
Actual value of P: 71


# Schritt 4

In [509]:
# Potential to optimize when N/P not hole Number
exp_Q = int(pub_User["N"]//exp_P)

exp_alt_Q = int(pub_User["N"]//exp_alt_P)

print(exp_Q, exp_alt_Q)
print("Actual value of Q:", User["Q"])

5107 1597
Actual value of Q: 5107


## Schritt 5

In [510]:
exp_PhiN = int(calcPhiN(exp_P, exp_Q))

exp_alt_PhiN = int(calcPhiN(exp_alt_P, exp_alt_Q))

print(exp_PhiN, exp_alt_PhiN)
print("Actual value of PhiN:", User["PhiN"])

exp_D = mmi(pub_User["E"], exp_PhiN)

exp_alt_D = mmi(pub_User["E"], exp_alt_PhiN)

print(exp_D, exp_alt_D)
print("Actual value of D:", User["D"])

357420 360696
Actual value of PhiN: 357420
147173 254609
Actual value of D: 147173


## Schritt 6

In [511]:
mal_signature = pow(msg, exp_D, User["N"])
alt_mal_signature = pow(msg, exp_alt_D, User["N"])

if(signature == mal_signature):
    print("D is the right privat Key:",exp_D)
elif(signature == alt_mal_signature):
    print("D' is the right privat Key:",exp_alt_D)


D is the right privat Key: 147173
